# ITR Schedule FA Automated Data Entry
Automatically enter data from an excel file to the ITR Schedule FA in the ITR-2 form.

---

## Pre-requisites
- Download the webdriver for your browser and place it in some directory and keep a note of it. The notebook is currently configured to use the Chrome webdriver and will work for any Chromium based browser. Download the Chrome webdriver from [here](https://chromedriver.chromium.org/downloads).
- Following variables are **required to be set** before running the script:

| Variable | Description | Example |
| --- | --- | --- |
| `chrome_driver_path` | Local path to the downloaded webdriver **chromedriver.exe** file | r"C:\Files\chromedriver_win32\chromedriver.exe" |
| `browser_path` | Local path to the **chrome.exe** file | r"C:\Files\Google\Chrome\Application\chrome.exe" |
| `excel_file_path` | Path to the excel file containing Schedule FA details | r"C:\Files\itr-schedule-fa.xlsx" |
| `home_url` | URL of the ITR filing portal | Already defined below |
| `country` | Value for country that will be used in the Schedule FA form at UI  | Already defined below |

In [ ]:
chrome_driver_path = r"<PATH_TO_CHROME_DRIVER>"
browser_path = r"<PATH_TO_BROWSER>"
excel_file_path = r"<PATH_TO_SCHEDULE_FA_REPORT>"

home_url = "https://eportal.incometax.gov.in"
country = "2 - UNITED STATES"

## Read Excel File
The following cell reads in the data from the excel file and stores it in a dataframe named `df`.

In [ ]:
import pandas as pd

month_map = {
    "01": "Jan",
    "02": "Feb",
    "03": "Mar",
    "04": "Apr",
    "05": "May",
    "06": "Jun",
    "07": "Jul",
    "08": "Aug",
    "09": "Sep",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec",
}
df = pd.read_excel(excel_file_path)
df.head()

## Preprocess Data
Make sure that the data in the excel file does not contain any null values. If there are any null values, replace them with appropriate values. Also, go through the data and do your own sanity checks to ensure that the data is correct and good to go.

In [ ]:
# check null values in df
df.isnull().sum()

## Open the ITR home page
After the following cell is executed, you will see the `home_url` opened in your browser.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep


def get_driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    options.binary_location = browser_path
    driver = webdriver.Chrome(chrome_driver_path, options=options)
    driver.maximize_window()
    return driver


driver = get_driver()
driver.get(home_url)

Now you need to do the following steps **manually** before running the next cell in the opened browser window:
- Login to the ITR portal.
- Complete the steps and open the **Schedule FA** page in the ITR-2 form.
- Make sure to do the first entry manually. The script will start entering data from the second row onwards.
- Make sure your page looks like this before running the next cell:<br/>
![Schedule FA](../images/schedule-fa-pre-requisite.png)

> Note: Notice that the **A3** section in the above image is collapsed. Make sure you have the same state in your web browser window. This is important because the section gets auto collpased when you succesfuly add a row to this section.

## Copy data from dataframe to the UI
The following cell iterates over the dataframe `df` row by row and does the following for each row:
- fetches values from the columns (you will need to **modify** the column names based on what you have in the excel file)
- gets the UI elements for the corresponding columns
- fills the fetched values in the correspoding fetched UI elements
- prints the row number after the row is successfully copied (this is helpful in case you want to run the cell again after some error, set `idx_start` to the next row number that was printed in the output)

The cell will take time dependiong on the number of rows in your excel file. Please be patient.

> Note: The variable `idx_start` is used to specify the row number from which the script should start entering data. This is useful in case you want to run the script again after some error. In that case, you can set the value of `idx_start` to the row number from which you want to start entering data.

In [ ]:
idx_start = 1

for idx in range(idx_start, len(df)):

    # get values for all fields from df
    row = df.iloc[idx]
    name_of_entity = row["Name of entity"]
    address_of_entity = row["Address of entity"]
    zip_code = row["zip code"]
    nature_of_entity = row["Nature of entity"]
    date = str(row["Date of acquiring interest/ stake"])
    init_value = str(row["Initial value of investment"])
    peak_value = str(row["Peak value of investment"])
    closing_value = str(row["Closing Balance"])
    gross_amount = str(
        row[
            "Total gross amount paid/credited with respect to the holding during the period"
        ]
    )
    gross_proceeds = str(
        row[
            "Total gross proceeds from sale or redemption of investment during the period"
        ]
    )

    # click on 'A3' section
    a3_section = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_hyper_490"]/div/div[1]/div/div/span'
    )
    a3_section.click()

    sleep(2)

    # click on 'Add Another' button
    add_another_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_button_330"]'
    )
    add_another_elem.click()

    # get all elements from UI
    country_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_dropdown_164"]'
    )
    name_of_entity_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_203"]'
    )
    address_of_entity_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_204"]'
    )
    zip_code_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_205"]'
    )
    nature_of_entity_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_206"]'
    )
    date_elem = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_textbox_471"]')
    init_value_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_207"]'
    )
    peak_value_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_208"]'
    )
    closing_value_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_209"]'
    )
    gross_amount_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_210"]'
    )
    gross_proceeds_elem = driver.find_element(
        By.XPATH, '//*[@id="uniservenxtcmp_textbox_211"]'
    )
    add_elem = driver.find_element(By.XPATH, '//*[@id="uniservenxtcmp_button_279"]')

    # Put value into country dropdown at UI
    country_select = Select(country_elem)
    sleep(2)
    country_select.select_by_visible_text(country)

    # Put value into all other elements at UI
    name_of_entity_elem.send_keys(name_of_entity)
    address_of_entity_elem.send_keys(address_of_entity)
    zip_code_elem.send_keys(zip_code)
    nature_of_entity_elem.send_keys(nature_of_entity)
    init_value_elem.send_keys(init_value)
    peak_value_elem.send_keys(peak_value)
    closing_value_elem.send_keys(closing_value)
    gross_amount_elem.send_keys(gross_amount)
    gross_proceeds_elem.send_keys(gross_proceeds)

    # Put value into date element at UI
    date_arr = date.split("-")
    year, month, day = date_arr[0], month_map[date_arr[1]], date_arr[2]
    date_elem.click()
    month_year_elem = driver.find_element(
        By.XPATH, "/html/body/div[5]/div[1]/table/thead/tr[2]/th[1]"
    )
    month_year_elem.click()
    year_elem = driver.find_element(
        By.XPATH, "/html/body/div[5]/div[2]/table/thead/tr[2]/th[1]"
    )
    year_elem.click()
    year_box = driver.find_element(By.XPATH, f"//*[text()='{year}']")
    year_box.click()
    month_box = driver.find_element(By.XPATH, f"//*[text()='{month}']")
    month_box.click()
    day_box = driver.find_element(
        By.XPATH, f"//td[@class='day' and text()='{int(day)}']"
    )
    sleep(1)
    day_box.click()

    # click on 'Add' button to add all the filled details
    add_elem.click()

    print(f"idx: {idx} copied successfully.")
    sleep(1)

print("-------------Task Completed-------------")

## Troubleshooting
Whenever the above cell fails, firstly you **must make sure that the page is back to the same state** as [mentioned earlier](../images/schedule-fa-pre-requisite.png) and then you should do the following in order until the issue is resolved:
- The cell will print the last row number that was successful. You can set the value of `idx_start` to the last succesful row number + 1 and run the cell again. This will start entering data from the row where it failed.
- If you see the same error again, identify the line where it fails. Put a `time.sleep(2)` before the failed line if it's not already there or try increasing the value of `time.sleep()` and run the cell again.

> Note: If you can't resolve the error, feel free to create an issue on the [GitHub repository](https://github.com/iamrk04/DataEntryAssistant/issues). Make sure to include the error message with the entire stacktrace.

## Disclaimer
- This is a personal project and is not affiliated with the Income Tax Department of India in any way.
- Make sure to **verify the data entered** by the script before submitting the form.
- The author does not make any warranty, express or implied, or assume any legal/consequential liability, or responsibility for the authenticity, accuracy and completeness of the data filled at website.
- The author is not responsible for any loss or damage caused by the use of this software.